# Multi Day Model

Combining rainfall data and stream flow make the model better at predicting extreme changes in water level. Now we will input multiple days worth of data into the neural network to predict the next day.

In [1]:
# imports, setup and helper functions
import numpy as np
import pandas as pd
import altair as alt
import requests
import json
from datetime import datetime
import pprint
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

water_level_df = pd.read_csv("datasets/warragamba_dam_level_raw_2008_2022.csv", index_col=0)
rainfall_df = pd.read_csv("datasets/rainfall_within_0.2_lat_degrees_of_warragamba_dam_2008_2022.csv", index_col=0)
stream_df = pd.read_csv("datasets/cox_and_wollon_stream_level_2008-2022.csv", index_col=0)

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# clean water_level_df so that levels with quality code 201 or 255 are set to NaN

print(water_level_df["q"].unique())
water_level_df.loc[water_level_df["q"].isin([201,255]), "v"] = np.NaN

water_level_df

[255   4 201   6  10  51  11 155   5   2   8]


,v,t,q
0,NaN,20080130000000,255
1,-13.588,20080131000000,4
2,-13.573,20080201000000,4
3,-13.564,20080202000000,4
4,-13.545,20080203000000,4
...,...,...,...
5092,-0.300,20220108000000,8
5093,0.086,20220109000000,8
5094,0.049,20220110000000,8
5095,0.028,20220111000000,8


In [ ]:
# rename df columns to prepare for merge into main_df
water_level_col = "v_212242_130"
water_level_df = water_level_df.rename(columns={"v":"v_212242_130", "q":"q_212242_130"})

In [ ]:
# join water_level and rainfall df by timestamp
df = pd.merge(
        left=water_level_df,
        right=rainfall_df,
        how="inner",
        on="t"
    )

# join stream df to main df
df = pd.merge(
        left=df,
        right=stream_df,
        how="inner",
        on="t"
    )

df

,v_212242_130,t,q_212242_130,v_563035_10,q_563035_10,v_563046_10,q_563046_10,v_563079_10,q_563079_10,v_568045_10,q_568045_10,v_568051_10,q_568051_10,v_212250_100,q_212250_100,v_212270_100,q_212270_100
0,NaN,20080130000000,255,1.5,4,4.5,4,0.0,100,0.5,4,2.0,4,0.205,4,0.651,6
1,-13.588,20080131000000,4,4.5,4,1.5,4,15.5,100,2.5,4,15.0,4,0.266,4,0.742,6
2,-13.573,20080201000000,4,3.0,4,9.5,4,9.0,100,0.0,4,2.0,4,0.350,4,0.742,6
3,-13.564,20080202000000,4,0.0,4,0.5,4,0.5,100,0.0,4,0.0,4,0.288,4,0.727,6
4,-13.545,20080203000000,4,17.0,4,13.5,4,58.0,100,29.5,4,13.5,4,0.250,4,0.735,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091,-0.723,20220107000000,8,39.5,1,46.0,1,39.5,1,30.5,1,41.0,1,0.950,6,1.154,1
5092,-0.300,20220108000000,8,2.5,1,0.5,1,18.0,1,5.0,1,1.0,1,1.320,6,3.077,1
5093,0.086,20220109000000,8,0.0,1,2.0,1,1.5,1,0.5,1,2.5,1,1.128,6,3.066,1
5094,0.049,20220110000000,8,0.5,1,1.0,1,1.0,1,1.5,1,1.5,1,0.931,6,2.128,1


In [ ]:
water_level = df[water_level_col]
water_level_tomorrow = df[water_level_col].copy().shift(1, fill_value=0.0)

df["water_level_difference"] = water_level - water_level_tomorrow
df

,v_212242_130,t,q_212242_130,v_563035_10,q_563035_10,v_563046_10,q_563046_10,v_563079_10,q_563079_10,v_568045_10,q_568045_10,v_568051_10,q_568051_10,v_212250_100,q_212250_100,v_212270_100,q_212270_100,water_level_difference
0,NaN,20080130000000,255,1.5,4,4.5,4,0.0,100,0.5,4,2.0,4,0.205,4,0.651,6,NaN
1,-13.588,20080131000000,4,4.5,4,1.5,4,15.5,100,2.5,4,15.0,4,0.266,4,0.742,6,NaN
2,-13.573,20080201000000,4,3.0,4,9.5,4,9.0,100,0.0,4,2.0,4,0.350,4,0.742,6,0.015
3,-13.564,20080202000000,4,0.0,4,0.5,4,0.5,100,0.0,4,0.0,4,0.288,4,0.727,6,0.009
4,-13.545,20080203000000,4,17.0,4,13.5,4,58.0,100,29.5,4,13.5,4,0.250,4,0.735,6,0.019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091,-0.723,20220107000000,8,39.5,1,46.0,1,39.5,1,30.5,1,41.0,1,0.950,6,1.154,1,0.051
5092,-0.300,20220108000000,8,2.5,1,0.5,1,18.0,1,5.0,1,1.0,1,1.320,6,3.077,1,0.423
5093,0.086,20220109000000,8,0.0,1,2.0,1,1.5,1,0.5,1,2.5,1,1.128,6,3.066,1,0.386
5094,0.049,20220110000000,8,0.5,1,1.0,1,1.0,1,1.5,1,1.5,1,0.931,6,2.128,1,-0.037


In [ ]:
# add water level difference tomorrow and water level in 2 days time
df["water_level_difference_plus_1"] = df["water_level_difference"].copy().shift(-1, fill_value=0.0)
df["water_level_difference_plus_2"] = df["water_level_difference"].copy().shift(-2, fill_value=0.0)
df[["water_level_difference", "water_level_difference_plus_1", "water_level_difference_plus_2"]]

,water_level_difference,water_level_difference_plus_1,water_level_difference_plus_2
0,NaN,NaN,0.015
1,NaN,0.015,0.009
2,0.015,0.009,0.019
3,0.009,0.019,0.054
4,0.019,0.054,0.169
...,...,...,...
5091,0.051,0.423,0.386
5092,0.423,0.386,-0.037
5093,0.386,-0.037,-0.021
5094,-0.037,-0.021,0.000


In [ ]:
# if corresponding q_ (quality column) 201 or 255, set v_ column to 0.0

value_cols = df.copy().filter(regex="^v_").columns.to_list()
print(value_cols)
df.columns

['v_212242_130', 'v_563035_10', 'v_563046_10', 'v_563079_10', 'v_568045_10', 'v_568051_10', 'v_212250_100', 'v_212270_100']


Index(['v_212242_130', 't', 'q_212242_130', 'v_563035_10', 'q_563035_10',
       'v_563046_10', 'q_563046_10', 'v_563079_10', 'q_563079_10',
       'v_568045_10', 'q_568045_10', 'v_568051_10', 'q_568051_10',
       'v_212250_100', 'q_212250_100', 'v_212270_100', 'q_212270_100',
       'water_level_difference', 'water_level_difference_plus_1',
       'water_level_difference_plus_2'],
      dtype='object')

In [ ]:
quality_cols = df.filter(regex="^q_").columns.to_list()
print(quality_cols)

['q_212242_130', 'q_563035_10', 'q_563046_10', 'q_563079_10', 'q_568045_10', 'q_568051_10', 'q_212250_100', 'q_212270_100']


In [ ]:
for value_col, quality_col in zip(value_cols, quality_cols):
    df.loc[df[quality_col].isin([201, 255]), value_col] = 0.0
    
df

,v_212242_130,t,q_212242_130,v_563035_10,q_563035_10,v_563046_10,q_563046_10,v_563079_10,q_563079_10,v_568045_10,q_568045_10,v_568051_10,q_568051_10,v_212250_100,q_212250_100,v_212270_100,q_212270_100,water_level_difference,water_level_difference_plus_1,water_level_difference_plus_2
0,0.000,20080130000000,255,1.5,4,4.5,4,0.0,100,0.5,4,2.0,4,0.205,4,0.651,6,NaN,NaN,0.015
1,-13.588,20080131000000,4,4.5,4,1.5,4,15.5,100,2.5,4,15.0,4,0.266,4,0.742,6,NaN,0.015,0.009
2,-13.573,20080201000000,4,3.0,4,9.5,4,9.0,100,0.0,4,2.0,4,0.350,4,0.742,6,0.015,0.009,0.019
3,-13.564,20080202000000,4,0.0,4,0.5,4,0.5,100,0.0,4,0.0,4,0.288,4,0.727,6,0.009,0.019,0.054
4,-13.545,20080203000000,4,17.0,4,13.5,4,58.0,100,29.5,4,13.5,4,0.250,4,0.735,6,0.019,0.054,0.169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091,-0.723,20220107000000,8,39.5,1,46.0,1,39.5,1,30.5,1,41.0,1,0.950,6,1.154,1,0.051,0.423,0.386
5092,-0.300,20220108000000,8,2.5,1,0.5,1,18.0,1,5.0,1,1.0,1,1.320,6,3.077,1,0.423,0.386,-0.037
5093,0.086,20220109000000,8,0.0,1,2.0,1,1.5,1,0.5,1,2.5,1,1.128,6,3.066,1,0.386,-0.037,-0.021
5094,0.049,20220110000000,8,0.5,1,1.0,1,1.0,1,1.5,1,1.5,1,0.931,6,2.128,1,-0.037,-0.021,0.000


In [ ]:
# remove quality columns
df = df.loc[:, ~df.columns.str.contains('^q_')]

df

,v_212242_130,t,v_563035_10,v_563046_10,v_563079_10,v_568045_10,v_568051_10,v_212250_100,v_212270_100,water_level_difference,water_level_difference_plus_1,water_level_difference_plus_2
0,0.000,20080130000000,1.5,4.5,0.0,0.5,2.0,0.205,0.651,NaN,NaN,0.015
1,-13.588,20080131000000,4.5,1.5,15.5,2.5,15.0,0.266,0.742,NaN,0.015,0.009
2,-13.573,20080201000000,3.0,9.5,9.0,0.0,2.0,0.350,0.742,0.015,0.009,0.019
3,-13.564,20080202000000,0.0,0.5,0.5,0.0,0.0,0.288,0.727,0.009,0.019,0.054
4,-13.545,20080203000000,17.0,13.5,58.0,29.5,13.5,0.250,0.735,0.019,0.054,0.169
...,...,...,...,...,...,...,...,...,...,...,...,...
5091,-0.723,20220107000000,39.5,46.0,39.5,30.5,41.0,0.950,1.154,0.051,0.423,0.386
5092,-0.300,20220108000000,2.5,0.5,18.0,5.0,1.0,1.320,3.077,0.423,0.386,-0.037
5093,0.086,20220109000000,0.0,2.0,1.5,0.5,2.5,1.128,3.066,0.386,-0.037,-0.021
5094,0.049,20220110000000,0.5,1.0,1.0,1.5,1.5,0.931,2.128,-0.037,-0.021,0.000


In [ ]:
# Copy value columns, shift down by 1 and join to data frame.
# This will mean each column will have data for the current day and the previous day.

yesterday_values = df[value_cols].copy().shift(1, fill_value=0.0)
day_minus_2_values = df[value_cols].copy().shift(2, fill_value=0.0)


yesterday_columns = list(map(lambda c: f"{c}_dm1", value_cols))
day_minus_2_columns = list(map(lambda c: f"{c}_dm2", value_cols))

yesterday_values = yesterday_values.rename(columns=dict(zip(value_cols, yesterday_columns)))
day_minus_2_values = day_minus_2_values.rename(columns=dict(zip(value_cols, day_minus_2_columns)))

df = pd.concat([df, yesterday_values, day_minus_2_values], axis=1)
df

,v_212242_130,t,v_563035_10,v_563046_10,v_563079_10,v_568045_10,v_568051_10,v_212250_100,v_212270_100,water_level_difference,...,v_212250_100_dm1,v_212270_100_dm1,v_212242_130_dm2,v_563035_10_dm2,v_563046_10_dm2,v_563079_10_dm2,v_568045_10_dm2,v_568051_10_dm2,v_212250_100_dm2,v_212270_100_dm2
0,0.000,20080130000000,1.5,4.5,0.0,0.5,2.0,0.205,0.651,NaN,...,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000
1,-13.588,20080131000000,4.5,1.5,15.5,2.5,15.0,0.266,0.742,NaN,...,0.205,0.651,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000
2,-13.573,20080201000000,3.0,9.5,9.0,0.0,2.0,0.350,0.742,0.015,...,0.266,0.742,0.000,1.5,4.5,0.0,0.5,2.0,0.205,0.651
3,-13.564,20080202000000,0.0,0.5,0.5,0.0,0.0,0.288,0.727,0.009,...,0.350,0.742,-13.588,4.5,1.5,15.5,2.5,15.0,0.266,0.742
4,-13.545,20080203000000,17.0,13.5,58.0,29.5,13.5,0.250,0.735,0.019,...,0.288,0.727,-13.573,3.0,9.5,9.0,0.0,2.0,0.350,0.742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091,-0.723,20220107000000,39.5,46.0,39.5,30.5,41.0,0.950,1.154,0.051,...,0.483,0.795,-0.763,23.0,18.0,31.5,39.0,27.5,0.405,0.811
5092,-0.300,20220108000000,2.5,0.5,18.0,5.0,1.0,1.320,3.077,0.423,...,0.950,1.154,-0.774,10.5,18.0,34.0,29.0,44.5,0.483,0.795
5093,0.086,20220109000000,0.0,2.0,1.5,0.5,2.5,1.128,3.066,0.386,...,1.320,3.077,-0.723,39.5,46.0,39.5,30.5,41.0,0.950,1.154
5094,0.049,20220110000000,0.5,1.0,1.0,1.5,1.5,0.931,2.128,-0.037,...,1.128,3.066,-0.300,2.5,0.5,18.0,5.0,1.0,1.320,3.077


In [ ]:
# remove row if 'water_level_difference' NaN
df = df.loc[~df["water_level_difference"].isnull()]
df

,v_212242_130,t,v_563035_10,v_563046_10,v_563079_10,v_568045_10,v_568051_10,v_212250_100,v_212270_100,water_level_difference,...,v_212250_100_dm1,v_212270_100_dm1,v_212242_130_dm2,v_563035_10_dm2,v_563046_10_dm2,v_563079_10_dm2,v_568045_10_dm2,v_568051_10_dm2,v_212250_100_dm2,v_212270_100_dm2
2,-13.573,20080201000000,3.0,9.5,9.0,0.0,2.0,0.350,0.742,0.015,...,0.266,0.742,0.000,1.5,4.5,0.0,0.5,2.0,0.205,0.651
3,-13.564,20080202000000,0.0,0.5,0.5,0.0,0.0,0.288,0.727,0.009,...,0.350,0.742,-13.588,4.5,1.5,15.5,2.5,15.0,0.266,0.742
4,-13.545,20080203000000,17.0,13.5,58.0,29.5,13.5,0.250,0.735,0.019,...,0.288,0.727,-13.573,3.0,9.5,9.0,0.0,2.0,0.350,0.742
5,-13.491,20080204000000,55.5,58.0,216.5,86.5,117.5,0.310,0.721,0.054,...,0.250,0.735,-13.564,0.0,0.5,0.5,0.0,0.0,0.288,0.727
6,-13.322,20080205000000,2.0,2.5,2.5,18.0,3.5,0.495,0.728,0.169,...,0.310,0.721,-13.545,17.0,13.5,58.0,29.5,13.5,0.250,0.735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091,-0.723,20220107000000,39.5,46.0,39.5,30.5,41.0,0.950,1.154,0.051,...,0.483,0.795,-0.763,23.0,18.0,31.5,39.0,27.5,0.405,0.811
5092,-0.300,20220108000000,2.5,0.5,18.0,5.0,1.0,1.320,3.077,0.423,...,0.950,1.154,-0.774,10.5,18.0,34.0,29.0,44.5,0.483,0.795
5093,0.086,20220109000000,0.0,2.0,1.5,0.5,2.5,1.128,3.066,0.386,...,1.320,3.077,-0.723,39.5,46.0,39.5,30.5,41.0,0.950,1.154
5094,0.049,20220110000000,0.5,1.0,1.0,1.5,1.5,0.931,2.128,-0.037,...,1.128,3.066,-0.300,2.5,0.5,18.0,5.0,1.0,1.320,3.077


In [ ]:
# separate training data from testing data
# separate features from labels

train_data = df.copy().sample(frac=0.8, random_state=12345)
test_data = df.copy().drop(train_data.index)

train_features = train_data.copy()
test_features = test_data.copy()

train_labels = train_features.pop("water_level_difference")
test_labels = test_features.pop("water_level_difference")

In [ ]:
feature_columns = train_features.filter(regex="^v_.*_(100|10).*$").columns

In [ ]:
# created normaliser which will normalise input via (input - mean) / sqrt(var)
np_train_features = train_features[feature_columns]
np_train_labels = train_labels.values
np_train_labels = np_train_labels.reshape(-1, 1)

np_test_features = test_features[feature_columns].values
np_test_labels = test_labels.values

normaliser = tf.keras.layers.Normalization(axis=1)
normaliser.adapt(np_train_features)

display(np_train_features)


,v_563035_10,v_563046_10,v_563079_10,v_568045_10,v_568051_10,v_212250_100,v_212270_100,v_563035_10_dm1,v_563046_10_dm1,v_563079_10_dm1,...,v_568051_10_dm1,v_212250_100_dm1,v_212270_100_dm1,v_563035_10_dm2,v_563046_10_dm2,v_563079_10_dm2,v_568045_10_dm2,v_568051_10_dm2,v_212250_100_dm2,v_212270_100_dm2
871,0.0,2.0,0.0,0.0,0.0,0.161,0.407,0.0,0.0,0.0,...,0.0,0.165,0.415,0.0,0.0,0.0,0.0,0.0,0.167,0.436
4110,0.0,0.0,0.0,0.0,0.0,0.063,0.249,0.0,0.0,0.0,...,0.0,0.066,0.251,0.0,0.0,0.0,0.0,0.0,0.065,0.253
3248,0.0,0.0,2.5,1.0,2.5,0.179,0.349,0.0,0.0,0.0,...,1.0,0.215,0.347,0.0,0.0,0.0,0.0,0.0,0.253,0.323
3699,0.0,0.0,0.0,0.0,0.0,0.085,0.374,0.0,0.0,0.0,...,0.0,0.090,0.388,0.0,0.0,0.0,0.0,0.0,0.091,0.399
4999,0.0,0.0,0.0,0.0,0.0,0.246,0.828,0.0,0.0,0.0,...,0.0,0.268,0.867,0.0,0.0,0.0,0.0,0.0,0.295,0.749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3303,0.0,0.0,2.0,0.0,0.0,0.045,0.412,0.0,0.0,0.0,...,0.0,0.053,0.334,0.0,0.0,0.0,0.0,0.0,0.066,0.040
2599,0.0,1.5,0.0,0.0,0.0,0.106,0.410,8.5,2.0,0.0,...,1.5,0.107,0.421,14.5,7.0,0.0,9.0,17.0,0.102,0.423
1085,0.0,2.5,16.5,11.0,0.0,0.353,0.654,0.0,1.5,5.0,...,0.0,0.376,0.673,0.0,0.0,0.0,0.0,0.0,0.413,0.703
56,0.5,0.5,0.5,0.0,0.0,0.245,0.471,17.5,22.0,22.5,...,19.0,0.290,0.466,25.5,31.5,7.0,2.5,29.5,0.160,0.467


In [ ]:
test_model = tf.keras.Sequential(name="stream_rain_temporal_model", layers=[
    layers.Input(shape=(21,)),
    normaliser,
    layers.Dropout(rate=0.5),
    layers.Dense(units=32, activation="relu"),
    layers.Dropout(rate=0.5),
    layers.Dense(units=1)
])

test_model.summary()

Model: "stream_rain_temporal_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_56 (Normaliza  (None, 21)               43        
 tion)                                                           
                                                                 
 dropout_110 (Dropout)       (None, 21)                0         
                                                                 
 dense_118 (Dense)           (None, 32)                704       
                                                                 
 dropout_111 (Dropout)       (None, 32)                0         
                                                                 
 dense_119 (Dense)           (None, 32)                1056      
                                                                 
 dropout_112 (Dropout)       (None, 32)                0         
                                        

In [ ]:
# check it allows input that is the same shape as the features
test_model.predict(np_train_features[0:1])

1/1 [==============================] - 0s 492ms/step


array([[-0.0056265]], dtype=float32)

In [ ]:
test_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_absolute_error')

In [ ]:
history = test_model.fit(
    np_train_features,
    np_train_labels,
    epochs=50,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

Epoch 1/50
93/93 [==============================] - 1s 3ms/step - loss: 0.3884 - val_loss: 0.0657
Epoch 2/50
93/93 [==============================] - 0s 2ms/step - loss: 0.1908 - val_loss: 0.0358
Epoch 3/50
93/93 [==============================] - 0s 2ms/step - loss: 0.1278 - val_loss: 0.0265
Epoch 4/50
93/93 [==============================] - 0s 1ms/step - loss: 0.0806 - val_loss: 0.0217
Epoch 5/50
93/93 [==============================] - 0s 2ms/step - loss: 0.0610 - val_loss: 0.0206
Epoch 6/50
93/93 [==============================] - 0s 2ms/step - loss: 0.0502 - val_loss: 0.0201
Epoch 7/50
93/93 [==============================] - 0s 2ms/step - loss: 0.0435 - val_loss: 0.0199
Epoch 8/50
93/93 [==============================] - 0s 2ms/step - loss: 0.0395 - val_loss: 0.0200
Epoch 9/50
93/93 [==============================] - 0s 2ms/step - loss: 0.0375 - val_loss: 0.0193
Epoch 10/50
93/93 [==============================] - 0s 2ms/step - loss: 0.0317 - val_loss: 0.0196
Epoch 11/50
93/93 [

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df["epoch"] = history.epoch

alt.Chart(hist_df).mark_line().encode(
    x="epoch:Q",
    y="loss:Q"
) + alt.Chart(hist_df).mark_line(color="orange").encode(
    x="epoch:Q",
    y="val_loss:Q"
)

/Users/chris/Library/Caches/pypoetry/virtualenvs/warragamba-watergate-M4NS-WVc-py3.10/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [ ]:
test_model.evaluate(np_test_features, np_test_labels)

29/29 [==============================] - 0s 1ms/step - loss: 0.0204


0.020410189405083656

In [ ]:
# compare predictions to real values of training set

y = test_model.predict(df[feature_columns].values)

compare_df = pd.DataFrame({
    "t": df[["t"]].values.flatten(),
    "actual": df[["water_level_difference"]].values.flatten(),
    "prediction": y.flatten()
})

compare_df.describe()

145/145 [==============================] - 0s 860us/step


,t,actual,prediction
count,4.626000e+03,4626.000000,4626.000000
mean,2.014874e+13,0.003181,-0.007416
std,4.037904e+10,0.139351,0.017987
min,2.008020e+13,-0.585000,-0.018553
25%,2.011121e+13,-0.020000,-0.013516
50%,2.015062e+13,-0.010000,-0.010304
75%,2.018111e+13,0.002000,-0.007930
max,2.022011e+13,6.774000,0.443791


In [ ]:
base = alt.Chart(compare_df.reset_index()[0:5000]).encode(
    x="index:Q"
)

(base.mark_line().encode(
    y="actual:Q"
) + base.mark_line(color="orange").encode(
    y="prediction:Q"
)).interactive()

/Users/chris/Library/Caches/pypoetry/virtualenvs/warragamba-watergate-M4NS-WVc-py3.10/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)